In [ ]:
import osmnx as ox
import pandas as pd
import pvlib 
from datetime import datetime, date, time, timedelta 

In [ ]:
superf_techo = {'surface_tilt': 0,'surface_azimuth': 180} #Inclinación y acimut de la superficie inclinada
superf_derecha = {'surface_tilt': 90,'surface_azimuth': 90} #Inclinación y acimut de la superficie inclinada
superf_izquierda = {'surface_tilt': 90,'surface_azimuth': 270} #Inclinación y acimut de la superficie inclinada
superf_capo = {'surface_tilt': 15,'surface_azimuth': 0} #Inclinación y acimut de la superficie inclinada
superf_maletero = {'surface_tilt': 45,'surface_azimuth': 180} #Inclinación y acimut de la superficie inclinada
coord_origen = (40.405551, -3.700684 ) #ETSIDI
coord_destino = (40.419120, -3.693025 )#Cibeles
timezone = 'Etc/GMT+2'
altitude=667
hora_salida = datetime(2020, 7, 28, 15, 7, 35) #y, m, d, h, min, s 

In [ ]:
superficies_coche =[]
superficies_coche.append(superf_techo)
superficies_coche.append(superf_derecha)
superficies_coche.append(superf_izquierda)
superficies_coche.append(superf_capo)
superficies_coche.append(superf_maletero)
len(superficies_coche)

In [ ]:
superficies_coche[4]['surface_tilt']

In [ ]:
mapa_nodos = ox.graph_from_point (coord_origen, dist=7000, dist_type ='bbox' , network_type = 'drive')
fig, ax = ox.plot_graph(mapa_nodos) 
ox.speed.add_edge_speeds(mapa_nodos , hwy_speeds = 30 , fallback = 30 , precision = 0 )
ox.speed.add_edge_travel_times( mapa_nodos , precision = 0 )

In [ ]:
# #GRAFICAR A GEODATAFRAME
nodos, caminos = ox.graph_to_gdfs(mapa_nodos) #Función de OSMnx que convierte el multidigraph en geodataframes de NODOS y LÍNEAS
# pd.set_option("max_columns", None) #Showing only two columns
# pd.set_option("max_rows", None)

In [ ]:
#------------------SECCIÓN INFORMATIVA-----------------------------#
nodos.head() #GeoDataFrames de los nodos
caminos.head()

In [ ]:
#---------------------NODOS ORIGEN Y DESTINO SEGÚN COORDENADAS----------------------------------#
nodo_origen = ox.get_nearest_node(mapa_nodos, coord_origen, method='haversine', return_dist=False)
nodo_destino = ox.get_nearest_node(mapa_nodos, coord_destino, method='haversine', return_dist=False)
nodo_origen,nodo_destino

In [ ]:
#--------------------------NODOS DE LA RUTA---------------------------------------#
ruta = ox.shortest_path(G=mapa_nodos, orig=nodo_origen, dest=nodo_destino, weight='length')
print(ruta) #Obtenemos la lista de nodos que se encuentran en el camino más corto.
len(ruta)

In [ ]:
#--------------------------INFORMACIÓN CALLES DE LA RUTA ---------------------------------------#
lista_caminos =[]
for x in range(1,len(ruta)):
    lista_caminos.append(mapa_nodos[ruta[x-1]][ruta[x]])
len(lista_caminos)

In [ ]:
#--------------------------NOMBRE DE LAS CALLES---------------------------------------#
nombre_calles =[]
for x in range(0,len(lista_caminos)):
    nombre_calles.append(lista_caminos[x][0]['name'])
len(nombre_calles)

In [ ]:
#--------------------------NODOS INICIALES DE LAS CALLES---------------------------------------#
nodo1_calle =[]
for x in range(0,len(ruta)-1):
    nodo1_calle.append(ruta[x])
len(nodo1_calle)

In [ ]:
#--------------------------NODOS FINALES DE LAS CALLES---------------------------------------#
nodo2_calle =[]
for x in range(1,len(ruta)):
    nodo2_calle.append(ruta[x])
len(nodo2_calle)

In [ ]:
#--------------------------COORDENADAS TOTALES EN UNA LISTA---------------------------------------#
lista_coords_nodos =[]
for x in range(0,len(ruta)):
    lista_coords_nodos.append(mapa_nodos.nodes[ruta[x]]['y'])
    lista_coords_nodos.append(mapa_nodos.nodes[ruta[x]]['x'])
len(lista_coords_nodos)

In [ ]:
#--------------------------COORDENADAS FINALES DE LA RUTA---------------------------------------#
coords_nodos = []
for i in range(0, len(lista_coords_nodos), 2):
    coords_nodos.append(lista_coords_nodos[i:i+2])
len(coords_nodos)

In [ ]:
#--------------------------COORDENADAS INICIALES DE LAS CALLES---------------------------------------#
coords1_calle =[]
for x in range(0,len(ruta)-1): #Elimino la ultima coordenada para tener longitud 30
    coords1_calle.append(coords_nodos[x])
len(coords1_calle) 

In [ ]:
#-------------------------COORDENADAS FINALES DE LAS CALLES--------------------------------------#
coords2_calle =[]
for x in range(1,len(ruta)):
    coords2_calle.append(coords_nodos[x])
len(coords2_calle)

In [ ]:
#----------------------------TRAZADO DE LA RUTA-------------------------------------------#
caminos.plot(ax=ax, linewidth=1, edgecolor='dimgray') #Líneas
fig2, ax = ox.plot_graph_route(mapa_nodos, ruta)

In [ ]:
#-------------------------INFORMACIÓN POR NODO DE LA RUTA----------------------------------------#
ruta_info=ox.utils_graph.get_route_edge_attributes(mapa_nodos, ruta, attribute=None, minimize_key='length', retrieve_default=None)
ruta_info

In [ ]:
#----------------------------APROXIMACIÓN DE LOS SEGUNDOS-------------------------------------------#
time_aprox=[]
for x in range(0,len(ruta_info)):
    time_aprox.append(round(ruta_info[x]['travel_time']))

In [ ]:
#----------------------------VELOCIDAD POR CALLE EN KPH-------------------------------------------#
calles_speed =[]
for x in range(0,len(ruta_info)):
    calles_speed.append(ruta_info[x]['speed_kph'])
len(calles_speed)

In [ ]:
#---------HORA DE PASADA POR NODO-----------#
horas_intermedias = hora_salida
hora_llegada=hora_salida
horas_trayecto = []
for x in range(0,len(ruta_info)):
   horas_trayecto.append(horas_intermedias + timedelta(seconds=time_aprox[x]))
   horas_intermedias += timedelta(seconds=time_aprox[x])
for x in range(0,len(ruta_info)):
    hora_llegada+=timedelta(seconds=time_aprox[x])

In [ ]:
naive_times = pd.date_range(start=hora_salida, end=hora_llegada, freq='1S')
times = naive_times.tz_localize(timezone)
solar_pos = pvlib.solarposition.get_solarposition(times, coord_origen[0], coord_origen[1])

In [ ]:
#------NO SE TIENE EN CUENTA PARA CALCULOS-----#
aoi = []
for x in range(0,len (superficies_coche)):
    aoi.append(pvlib.irradiance.aoi(superficies_coche[x]['surface_tilt'], superficies_coche[x]['surface_azimuth'],
                               solar_pos['apparent_zenith'], solar_pos['azimuth']))


In [ ]:
dni_extra = pvlib.irradiance.get_extra_radiation(times)
airmass_rel = pvlib.atmosphere.get_relative_airmass(solar_pos['apparent_zenith'])
pressure = pvlib.atmosphere.alt2pres(altitude)
airmass_abs = pvlib.atmosphere.get_absolute_airmass(airmass_rel, pressure)
turbidez = pvlib.clearsky.lookup_linke_turbidity(times, coord_origen[0], coord_origen[1])
val_sol = pvlib.clearsky.ineichen(solar_pos['apparent_zenith'], airmass_abs, turbidez,
                                 dni_extra=dni_extra, altitude=altitude)

In [ ]:
irrad = []
for x in range(0,len (superficies_coche)):
    irrad.append(pvlib.irradiance.get_total_irradiance(superficies_coche[x]['surface_tilt'],
    superficies_coche[x]['surface_azimuth'], solar_pos['zenith'] , solar_pos['azimuth'], val_sol['dni'], val_sol['ghi'] , 
    val_sol['dhi'], dni_extra=dni_extra, airmass=airmass_abs, albedo=0.10,
    surface_type=None, model='isotropic', model_perez='allsitescomposite1990'))



In [ ]:
# import numpy as np
filtro_final=[]
filtro_irrad1=[]
filtro_irrad2=[]
filtro_irrad3=[]
filtro_irrad4=[]
filtro_irrad5=[]
for x in range(0,len(ruta_info)):
    filtro_irrad1.append(irrad[0].loc[horas_trayecto[x]])
    filtro_irrad2.append(irrad[1].loc[horas_trayecto[x]])
    filtro_irrad3.append(irrad[2].loc[horas_trayecto[x]])
    filtro_irrad4.append(irrad[3].loc[horas_trayecto[x]])
    filtro_irrad5.append(irrad[4].loc[horas_trayecto[x]])



In [ ]:
poa_global_filtro1=[] #Filtramos la parte de poa_global
poa_global_filtro2=[] #Filtramos la parte de poa_global
poa_global_filtro3=[] #Filtramos la parte de poa_global
poa_global_filtro4=[] #Filtramos la parte de poa_global
poa_global_filtro5=[] #Filtramos la parte de poa_global
for x in range(0,len(ruta_info)):
    poa_global_filtro1.append(filtro_irrad1[x]['poa_global'])
    poa_global_filtro2.append(filtro_irrad2[x]['poa_global'])
    poa_global_filtro3.append(filtro_irrad3[x]['poa_global'])
    poa_global_filtro4.append(filtro_irrad4[x]['poa_global'])
    poa_global_filtro5.append(filtro_irrad5[x]['poa_global'])


In [ ]:
#----------------------------------------DATAFRAME-------------------------------------------#
df = pd.DataFrame(nombre_calles,columns=['Dirección'])
df['km/h'] = calles_speed
df['Tiempo'] = time_aprox
df['Nodo Origen'] = nodo1_calle
df['Origen'] = coords1_calle
df['NodDestino'] = nodo2_calle
df['Destino'] = coords2_calle
df['Ruta'] = horas_trayecto
df['Techo'] = poa_global_filtro1
df['Derecha'] = poa_global_filtro2
df['Izquierda'] = poa_global_filtro3
df['Capó'] = poa_global_filtro4
df['Maletero'] = poa_global_filtro5

In [ ]:
print ('Fecha y hora de inicio de la ruta', hora_salida)
print('Fecha y hora de fin de la ruta', hora_llegada)
df.iloc[0:len(ruta)] # Primeras cinco filas
# pd.set_option("max_columns", None) #Showing only two columns
# pd.set_option("max_rows", None)